# BoxMOT 실행

[mikel-brostrom/yolo_tracking](https://github.com/mikel-brostrom/yolo_tracking) 저장소의 `tracking/track.py` 코드를 그대로 실행해보는 노트북 파일

`ultralytics` 라이브러리 사용과 동일한데, `on_predict_start` 상황에 실행되는 콜백 함수를 통해 트래커를 추가하는 것으로 보임

In [2]:
import argparse
from functools import partial

import torch

import boxmot
from boxmot import TRACKERS
from boxmot.tracker_zoo import create_tracker
from boxmot.utils import ROOT, WEIGHTS, TRACKER_CONFIGS
from boxmot.utils.checks import TestRequirements
from tracking.detectors import get_yolo_inferer

from ultralytics import YOLO

In [3]:
def parse_opt(argv):
    parser = argparse.ArgumentParser()
    parser.add_argument('--yolo-model', type=Path, default=WEIGHTS / 'yolov8n',
                        help='yolo model path')
    parser.add_argument('--reid-model', type=Path, default=WEIGHTS / 'osnet_x0_25_msmt17.pt',
                        help='reid model path')
    parser.add_argument('--tracking-method', type=str, default='deepocsort',
                        help='deepocsort, botsort, strongsort, ocsort, bytetrack')
    parser.add_argument('--source', type=str, default='0',
                        help='file/dir/URL/glob, 0 for webcam')
    parser.add_argument('--imgsz', '--img', '--img-size', nargs='+', type=int, default=[640],
                        help='inference size h,w')
    parser.add_argument('--conf', type=float, default=0.5,
                        help='confidence threshold')
    parser.add_argument('--iou', type=float, default=0.7,
                        help='intersection over union (IoU) threshold for NMS')
    parser.add_argument('--device', default='',
                        help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--show', action='store_true',
                        help='display tracking video results')
    parser.add_argument('--save', action='store_true',
                        help='save video tracking results')
    # class 0 is person, 1 is bycicle, 2 is car... 79 is oven
    parser.add_argument('--classes', nargs='+', type=int,
                        help='filter by class: --classes 0, or --classes 0 2 3')
    parser.add_argument('--project', default=ROOT / 'runs' / 'track',
                        help='save results to project/name')
    parser.add_argument('--name', default='exp',
                        help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true',
                        help='existing project/name ok, do not increment')
    parser.add_argument('--half', action='store_true',
                        help='use FP16 half-precision inference')
    parser.add_argument('--vid-stride', type=int, default=1,
                        help='video frame-rate stride')
    parser.add_argument('--show-labels', action='store_false',
                        help='either show all or only bboxes')
    parser.add_argument('--show-conf', action='store_false',
                        help='hide confidences when show')
    parser.add_argument('--show-trajectories', action='store_true',
                        help='show confidences')
    parser.add_argument('--save-txt', action='store_true',
                        help='save tracking results in a txt file')
    parser.add_argument('--save-id-crops', action='store_true',
                        help='save each crop to its respective id folder')
    parser.add_argument('--line-width', default=None, type=int,
                        help='The line width of the bounding boxes. If None, it is scaled to the image size.')
    parser.add_argument('--per-class', default=False, action='store_true',
                        help='not mix up classes when tracking')
    parser.add_argument('--verbose', default=True, action='store_true',
                        help='print results per frame')
    parser.add_argument('--agnostic-nms', default=False, action='store_true',
                        help='class-agnostic NMS')

    opt = parser.parse_args(argv)
    return opt

In [4]:
def on_predict_start(predictor, persist=False):
    """
    Initialize trackers for object tracking during prediction.

    Args:
        predictor (object): The predictor object to initialize trackers for.
        persist (bool, optional): Whether to persist the trackers if they already exist. Defaults to False.
    """
    print('ON_PREDICT_START')

    assert predictor.custom_args.tracking_method in TRACKERS, \
        f"'{predictor.custom_args.tracking_method}' is not supported. Supported ones are {TRACKERS}"

    tracking_config = TRACKER_CONFIGS / (predictor.custom_args.tracking_method + '.yaml')
    trackers = []
    for i in range(predictor.dataset.bs):
        tracker = create_tracker(
            predictor.custom_args.tracking_method,
            tracking_config,
            predictor.custom_args.reid_model,
            predictor.device,
            predictor.custom_args.half,
            predictor.custom_args.per_class
        )
        # motion only modeles do not have
        if hasattr(tracker, 'model'):
            tracker.model.warmup()
        trackers.append(tracker)

    predictor.trackers = trackers


@torch.no_grad()
def run(args):
    global yolo
    yolo = YOLO(
        args.yolo_model if 'yolov8' in str(args.yolo_model) else 'yolov8n.pt',
    )

    results = yolo.track(
        source=args.source,
        conf=args.conf,
        iou=args.iou,
        agnostic_nms=args.agnostic_nms,
        show=False,
        stream=True, # stream=False 로 하면 콜백 함수가 등록되지 않은 상태로 추론을 시작해서 트래커가 작동하지 않음
        device=args.device,
        show_conf=args.show_conf,
        save_txt=args.save_txt,
        show_labels=args.show_labels,
        save=args.save,
        verbose=args.verbose,
        exist_ok=args.exist_ok,
        project=args.project,
        name=args.name,
        classes=args.classes,
        imgsz=args.imgsz,
        vid_stride=args.vid_stride,
        line_width=args.line_width
    )

    # 트래커 관련 콜백 함수를 yolo.track 함수 호출 이후 등록함
    # yolo.track 호출 시 stream=False 인자를 넘겨주는데, 이 경우 results가 Generator 타입으로 반환되고,
    # 아래 반복문에서 실제 값을 얻어올 때 추론이 되는 것으로 보임
    yolo.add_callback('on_predict_start', partial(on_predict_start, persist=True))

    if 'yolov8' not in str(args.yolo_model):
        # replace yolov8 model
        m = get_yolo_inferer(args.yolo_model)
        model = m(
            model=args.yolo_model,
            device=yolo.predictor.device,
            args=yolo.predictor.args
        )
        yolo.predictor.model = model

    # store custom args in predictor
    yolo.predictor.custom_args = args

    ret = []
    for r in results:
        ret.append(r)

        img = yolo.predictor.trackers[0].plot_results(r.orig_img, args.show_trajectories)

        if args.show is True:
            cv2.imshow('BoxMOT', img)     
            key = cv2.waitKey(1) & 0xFF
            if key == ord(' ') or key == ord('q'):
                break
    return ret

In [5]:
argv = ['--classes', '0',                           # 0번 (사람) 클래스만 추적함
        '--yolo-model', 'yolov8n',                  # Detection 모델은 yolov8l 사용
        '--reid-model', 'osnet_x0_25_market1501.pt' # reid 모델은 osnet_x0_25_market1501.pt 사용
       ]
# 이 외 옵션들은 parse_opt 함수를 확인하거나, 저장소 README 파일 예제를 확인
# https://github.com/mikel-brostrom/yolo_tracking?tab=readme-ov-file#yolov8--yolo-nas--yolox-examples


opt = parse_opt(argv)

opt.source = '/home/kh/notebooks/assets/street.mp4' # 추론에 사용할 동영상 파일 경로
opt.save = True # 추론 결과를 저장함 (시각화)

In [6]:
results = run(opt)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6.23M/6.23M [00:00<00:00, 57.3MB/s]



ON_PREDICT_START


Downloading...
From: https://drive.google.com/uc?id=1z1UghYvOTtjx7kEoRfmqSMu-z62J6MAj
To: /home/kh/notebooks/240325월/osnet_x0_25_market1501.pt
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.46M/2.46M [00:00<00:00, 11.2MB/s]
2024-03-25 12:14:28.225 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_market1501.pt"


video 1/1 (1/1093) /home/kh/notebooks/assets/street.mp4: 384x640 6 persons, 118.9ms
video 1/1 (2/1093) /home/kh/notebooks/assets/street.mp4: 384x640 6 persons, 6.7ms
video 1/1 (3/1093) /home/kh/notebooks/assets/street.mp4: 384x640 4 persons, 8.2ms
video 1/1 (4/1093) /home/kh/notebooks/assets/street.mp4: 384x640 5 persons, 6.9ms
video 1/1 (5/1093) /home/kh/notebooks/assets/street.mp4: 384x640 6 persons, 6.9ms
video 1/1 (6/1093) /home/kh/notebooks/assets/street.mp4: 384x640 5 persons, 6.9ms
video 1/1 (7/1093) /home/kh/notebooks/assets/street.mp4: 384x640 6 persons, 6.9ms
video 1/1 (8/1093) /home/kh/notebooks/assets/street.mp4: 384x640 5 persons, 6.9ms
video 1/1 (9/1093) /home/kh/notebooks/assets/street.mp4: 384x640 5 persons, 7.3ms
video 1/1 (10/1093) /home/kh/notebooks/assets/street.mp4: 384x640 6 persons, 6.9ms
video 1/1 (11/1093) /home/kh/notebooks/assets/street.mp4: 384x640 6 persons, 6.9ms
video 1/1 (12/1093) /home/kh/notebooks/assets/street.mp4: 384x640 6 persons, 6.9ms
video 1/1 (

In [15]:
print(type(results))
print(type(results[0]))
print(len(results))

<class 'list'>
<class 'ultralytics.engine.results.Results'>
1093
